In [1]:
import requests
import arcpy
import os
import zipfile

In [10]:
zipfile_url = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/trans_road_centerlines_gac/shp_trans_road_centerlines_gac.zip'
download_directory = r'\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\Data'
extract_directory = r'\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\Data'

response = requests.get(zipfile_url)

if response.status_code == 200:
    zip_filename = os.path.basename(zipfile_url)
    zip_file_path = os.pa11 th.join(download_directory, zip_filename)

    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_directory)
    print(f'ZIP file extracted to: {extract_directory}')

os.remove(zip_file_path)

ZIP file extracted to: \\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\Data


In [ ]:
#Calculate Speeds and Distances
arcpy.management.CalculateField(
    in_table="RoadCenterline",
    field="Miles",
    expression=" nC",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="RoadCenterline",
    field="Minutes",
    expression="(!Miles!/!SPEED_IMP!)*60",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [12]:
#Create Feature Dataset For Network

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.CreateFeatureDataset(
        out_dataset_path=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb",
        out_name="MetroNetwork",
        spatial_reference='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]];-20037700 -30241100 10000;#;#;0.001;#;#;IsHighPrecision'
    )

In [13]:
# Add Centerlines to Network Geodatabse
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\Data\RoadCenterline.shp",
    Output_Geodatabase=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb\MetroNetwork"
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\FinalProjectStart\\MetroNetwork.gdb\\MetroNetwork'>

In [3]:
arcpy.na.CreateNetworkDataset(
    feature_dataset=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb\MetroNetwork",
    out_name="MetroNetworkOG",
    source_feature_class_names="RoadCenterline",
    elevation_model="ELEVATION_FIELDS"
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\FinalProjectStart\\FinalProjectStart.gdb\\Network\\NetworkWith94'>

In [20]:
arcpy.na.CreateTemplateFromNetworkDataset(
    network_dataset="MetroNetworkOG",
    output_network_dataset_template=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\BaseTemplate.xml"
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\FinalProjectStart\\BaseTemplate.xml'>

In [1]:
#Build the Network
arcpy.na.BuildNetwork(
    in_network_dataset=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb\MetroNetwork\MetroNetworkOG"
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\FinalProjectStart\\MetroNetwork.gdb\\MetroNetwork\\MetroNetworkOG'>

In [2]:
#Make Service Area Analysis Layer
n(
    network_data_source=r"\\Mac\Home\Documents\ArcGIS\Projects\FinalProjectStart\MetroNetwork.gdb\MetroNetwork\MetroNetworkOG",
    layer_name="Service Area",
    travel_mode="",
    travel_direction="FROM_FACILITIES",
    cutoffs=[5,10,20],
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    output_type="POLYGONS",
    polygon_detail="STANDARD",
    geometry_at_overlaps="OVERLAP",
    geometry_at_cutoffs="RINGS",
    polygon_trim_distance="10 Meters",
    exclude_sources_from_polygon_generation=None,
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

<Result 'Service Area'>

In [9]:
arcpy.na.AddLocations(
    in_network_analysis_layer="Service Area",
    sub_layer="Facilities",
    in_table="Facilities",
    field_mappings="Name Name #;CurbApproach # 0;Attr_Minutes # 0;Attr_Miles # 0;Breaks_Minutes # #;Breaks_Miles # #",
    search_tolerance="5000 Meters",
    sort_field=None,
    search_criteria="MetroCenterlines SHAPE;MetroNetworkOG_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="NO_SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

<Result 'Service Area'>

In [10]:
arcpy.na.Solve(
    in_network_analysis_layer="Service Area",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)

<Result 'Service Area'>